# 万方文献自动化下载脚本
## Reference
https://www.jianshu.com/p/83ab33037b84
## 使用说明
- 传入参数
  - search_expression(检索式),将要查询的检索式赋值给search_expression
  - download_path(默认下载地址),将想下载到的地址赋值给download_path,会修改浏览器默认下载位置
  - page_count(开始页数)，默认会从第page_count+1页位置处开始下载
- 目前的问题
  - 下载论文数量过多后会被封禁IP

In [7]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 配置检索式
search_expression = "(中图分类号:(R246.83) or 中图分类号:(R322.41) or 中图分类号:(R473.78) or 中图分类号:(R529.8) or 中图分类号:(R739.8) or 中图分类号:(R78) or 中图分类号:(R816.98) or 中图分类号:(R828.8) or 中图分类号:(R856.78) or 中图分类号:(R988.2) or 中图分类号:(TH787) or 中图分类号:(S853.58) or 中图分类号:(TQ658.4) not 刊名:(International Journal of Oral science) not 刊名:(中华口腔医学杂志) not 刊名:(华西口腔医学杂志) not 刊名:(临床口腔医学杂志) not 刊名:(国际口腔医学杂志) not 刊名:(中华老年口腔医学杂志) not 刊名:(中国实用口腔科杂志) not 刊名:(实用口腔医学杂志) not 刊名:(口腔疾病防治) not 刊名:(上海口腔医学) not 刊名:(口腔医学研究) not 刊名:(中华口腔正畸学杂志) not 刊名:(口腔医学) not 刊名:(口腔颌面修复学杂志) not 刊名:(中国口腔种植学杂志) not 刊名:(口腔材料器械杂志) not 刊名:(中国口腔颌面外科杂志) not 刊名:(北京口腔医学) not 刊名:(口腔生物医学) not 刊名:(口腔颌面外科杂志) not 刊名:(现代口腔医学杂志) not 刊名:(中华口腔医学研究杂志(电子版)) not 刊名:(全科口腔医学电子杂志) not 刊名:(牙体牙髓牙周病学杂志)) and 发表时间:2003-2024"

# 设置默认论文下载路径
download_path = "D:\\Desktop\\论文自动化下载\\万方论文"
#开始页码
page_count=51
#proxy = "your_proxy_ip:port"
# 设置浏览器选项
options = Options()
options.add_argument("--start-maximized")
#options.add_argument(f'--proxy-server={proxy}')
options.add_experimental_option("prefs", {
    "download.default_directory": download_path,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

# 启动浏览器
browser = webdriver.Edge(options=options)

# 打开网页
browser.get("https://s.wanfangdata.com.cn/advanced-search/paper")

# 执行检索
professional_search_button = browser.find_element(By.XPATH, "//span[text()='专业检索']")
professional_search_button.click()
text_input = browser.find_element(By.XPATH, "//*[@id='queryInput']/textarea")
text_input.send_keys(search_expression)
search_button = browser.find_element(By.XPATH, "/html/body/div[5]/div/div[2]/div[2]/div[1]/div[4]/span[1]")
search_button.click()
paper_count=0
for i in range(page_count):
    # 尝试点击“下一页”按钮
    next_button = WebDriverWait(browser, 3).until(
        EC.element_to_be_clickable((By.XPATH, "//span[@class='next' and text()='下一页']"))
    )
    browser.execute_script("arguments[0].scrollIntoView(true);", next_button)
    next_button.click()
    time.sleep(1)  # 等待页面加载
# 下载并翻页
while True:
    print("Downloading page", paper_count, "...")
    try:
        # 下载当前页的所有文档
        try:
            download_spans = WebDriverWait(browser, 3).until(
                EC.visibility_of_all_elements_located((By.XPATH, "//span[@data-v-9f00e464='' and text()='下载']"))
            )
            #print("Downloading", len(download_spans), "papers...")
        except Exception:
            download_spans = []
            #print("No papers found.")
        try:
            free_download_spans = WebDriverWait(browser, 3).until(
                EC.visibility_of_all_elements_located((By.XPATH, "//span[@data-v-e231288e='' and text()='下载']"))
            )
            #print("Downloading", len(free_download_spans), "free papers...")
        except Exception:
            free_download_spans = []
            #print("No free papers found.")
        paper_count+=len(download_spans)+len(free_download_spans)
        for span in download_spans + free_download_spans:
            try:
                browser.execute_script("arguments[0].scrollIntoView(true);", span)
                span.click()
                time.sleep(1)  # 等待下载开始

                # 切换到新窗口并关闭
                browser.switch_to.window(browser.window_handles[-1])
                browser.close()

                # 切换回原始窗口
                browser.switch_to.window(browser.window_handles[0])
            except Exception as e:
                print("Error clicking span or closing window:", e)
        
        try:
            # 尝试点击“下一页”按钮
            next_button = WebDriverWait(browser, 3).until(
                EC.element_to_be_clickable((By.XPATH, "//span[@class='next' and text()='下一页']"))
            )
            browser.execute_script("arguments[0].scrollIntoView(true);", next_button)
            next_button.click()
        except Exception:
            break
        time.sleep(1)  # 等待页面加载
    except Exception as e:
        print("No more pages or error:", e)
        break

# 关闭浏览器
browser.quit()


KeyboardInterrupt: 

# 万方论文摘要导出
## 使用说明
- 传入参数
  - search_expression(检索式),将要查询的检索式赋值给search_expression
  - download_path(默认下载地址),将想下载到的地址赋值给download_path,会修改浏览器默认下载位置
  - page_count(开始页数)，默认会从第page_count+1页位置处开始下载
- 目前的问题
  - 下载到5500条摘要后万方有查看限制，就不会展示论文列表，无法下载

In [6]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
# 配置检索式
search_expression = "(中图分类号:(R246.83) or 中图分类号:(R322.41) or 中图分类号:(R473.78) or 中图分类号:(R529.8) or 中图分类号:(R739.8) or 中图分类号:(R78) or 中图分类号:(R816.98) or 中图分类号:(R828.8) or 中图分类号:(R856.78) or 中图分类号:(R988.2) or 中图分类号:(TH787) or 中图分类号:(S853.58) or 中图分类号:(TQ658.4) not 刊名:(International Journal of Oral science) not 刊名:(中华口腔医学杂志) not 刊名:(华西口腔医学杂志) not 刊名:(临床口腔医学杂志) not 刊名:(国际口腔医学杂志) not 刊名:(中华老年口腔医学杂志) not 刊名:(中国实用口腔科杂志) not 刊名:(实用口腔医学杂志) not 刊名:(口腔疾病防治) not 刊名:(上海口腔医学) not 刊名:(口腔医学研究) not 刊名:(中华口腔正畸学杂志) not 刊名:(口腔医学) not 刊名:(口腔颌面修复学杂志) not 刊名:(中国口腔种植学杂志) not 刊名:(口腔材料器械杂志) not 刊名:(中国口腔颌面外科杂志) not 刊名:(北京口腔医学) not 刊名:(口腔生物医学) not 刊名:(口腔颌面外科杂志) not 刊名:(现代口腔医学杂志) not 刊名:(中华口腔医学研究杂志(电子版)) not 刊名:(全科口腔医学电子杂志) not 刊名:(牙体牙髓牙周病学杂志)) and 发表时间:2003-2024"

# 设置默认论文下载路径
download_path = "D:\\Desktop"
#开始页数
page_count=0#120
#proxy = "your_proxy_ip:port"
# 设置浏览器选项
options = Options()
options.add_argument("--start-maximized")
#options.add_argument(f'--proxy-server={proxy}')
options.add_experimental_option("prefs", {
    "download.default_directory": download_path,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

# 启动浏览器
browser = webdriver.Edge(options=options)

# 打开网页
browser.get("https://s.wanfangdata.com.cn/advanced-search/paper")

# 执行检索
professional_search_button = browser.find_element(By.XPATH, "//span[text()='专业检索']")
professional_search_button.click()
text_input = browser.find_element(By.XPATH, "//*[@id='queryInput']/textarea")
text_input.send_keys(search_expression)
search_button = browser.find_element(By.XPATH, "/html/body/div[5]/div/div[2]/div[2]/div[1]/div[4]/span[1]")
search_button.click()
# 等待下拉框可点击
dropdown = WebDriverWait(browser, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/div/div[1]/div[2]/div/div[1]/div[1]'))
)

browser.execute_script("arguments[0].click();", dropdown)

# 等待并选择特定的选项
option = WebDriverWait(browser, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/div/div[1]/div[2]/div/div[1]/div[2]/div[3]'))
)
browser.execute_script("arguments[0].click();", option)
paper_count=0

for i in range(page_count):
    # 尝试点击“下一页”按钮
    next_button = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[@class='next' and text()='下一页']"))
    )
    browser.execute_script("arguments[0].scrollIntoView(true);", next_button)
    next_button.click()
    time.sleep(1)  # 等待页面加载
time.sleep(5)  # 等待页面加载
# 下载并翻页
while True:
    print("Downloading page", paper_count, "...")
    for i in range (1,11):
        elements=browser.find_elements(By.XPATH, "/html/body/div[5]/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/div/div/label/span/input")
        for element in elements:
            browser.execute_script("arguments[0].scrollIntoView(true);", element)
            try:
                ActionChains(browser).move_to_element(element).click().perform()
            except Exception as e:
                print("Error clicking element:", e)
        try:
            # 尝试点击“下一页”按钮
            next_button = WebDriverWait(browser, 3).until(
                EC.element_to_be_clickable((By.XPATH, "//span[@class='next' and text()='下一页']"))
            )
            browser.execute_script("arguments[0].scrollIntoView(true);", next_button)
            next_button.click()
            time.sleep(3)  # 等待页面加载
        except Exception:
            break
    # 点击批量引用按钮
    export_button = browser.find_element(By.XPATH, "//span[text()='批量引用']")
    browser.execute_script("arguments[0].click();", export_button)
    original_handles = browser.window_handles
    WebDriverWait(browser, 30).until(lambda d: len(d.window_handles) > 1)
    # 切换到新窗口
    browser.switch_to.window(original_handles[-1])
    time.sleep(15)  # 等待页面加载
    tab_button = WebDriverWait(browser, 30).until(
        EC.element_to_be_clickable((By.XPATH, "//*[@id='tab-m3']"))
    )
    browser.execute_script("arguments[0].click();", tab_button)
    expport_button = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div[7]/div[2]/div/div/div[2]/div/div[1]/div[2]"))
    )
    try:
           ActionChains(browser).move_to_element(expport_button).click().perform()
    except Exception as e:
        print("Error clicking element:", e)
    time.sleep(1)  # 等待页面加载
    # 关闭当前窗口
    browser.close()

    # 切换回原始窗口
    browser.switch_to.window(original_handles[0])
    try:
        # 尝试点击“下一页”按钮
        next_button = WebDriverWait(browser, 3).until(
            EC.element_to_be_clickable((By.XPATH, "//span[@class='next' and text()='下一页']"))
        )
        browser.execute_script("arguments[0].scrollIntoView(true);", next_button)
        next_button.click()
    except Exception:
        break
    time.sleep(3)  # 等待页面加载
    clear_button = browser.find_element(By.XPATH, "/html/body/div[5]/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/div/div[1]/div[1]/div/span[2]")
    browser.execute_script("arguments[0].click();", clear_button)
    paper_count+=500
# 关闭浏览器
#browser.quit()


before open 1
after open 2
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2
before open 2
after open 3
after close 2


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[5]/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/div/div[1]/div[1]/div/span[2]"}
  (Session info: MicrosoftEdge=131.0.2903.146); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	(No symbol) [0x00007FF75D916B05]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF75DC3F444+1437348]
	sqlite3_dbdata_init [0x00007FF75DCE2D76+643174]
	(No symbol) [0x00007FF75D83C9DB]
	(No symbol) [0x00007FF75D83CAE3]
	(No symbol) [0x00007FF75D8792F7]
	(No symbol) [0x00007FF75D85C1DF]
	(No symbol) [0x00007FF75D833437]
	(No symbol) [0x00007FF75D876BFF]
	(No symbol) [0x00007FF75D85BE03]
	(No symbol) [0x00007FF75D832984]
	(No symbol) [0x00007FF75D831E30]
	(No symbol) [0x00007FF75D832571]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF75DBEBAD4+1094964]
	(No 

# 下载摘要格式化脚本
## 使用说明
- 将"./万方论文摘要"目录下的所有txt文件解析并保存成格式化的wanfang_abstract.json

In [33]:
process_path = "./万方论文摘要"
#依次遍历目录下的文件
import os
import json
json_list = []
for root, dirs, files in os.walk(process_path):
    for file in files:
        with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
            lines = f.readlines()
        # 解析文件内容
        abstracts=[]
        for i in range(0, len(lines), 4):
            if i + 2 < len(lines):
                entry =[lines[i].strip(), lines[i + 1].strip(), lines[i + 2].strip()]
                abstracts.append(entry)

        # 输出结果
        for abstract in abstracts:
            if len(abstract) < 3:
                continue
            reference = {
                "authors": abstract[0].split(". ")[0].split(" ")[1].strip(),
                "title": abstract[0].split(". ")[1].strip() if len(abstract[0].split(". "))>1 else "",
                "journal_info": abstract[0].split(". ")[2].strip() if len(abstract[0].split(". "))>2 else "",
                "degree_grantor": abstract[1].split(": ")[1].strip() if len(abstract[1].split(": "))>1 else "",
                "abstract": abstract[2].strip().replace("摘要: ", "")
            }
            json_list.append(reference)
            f.close()
# 将解析后的数据转换为 JSON 格式
json_data = json.dumps(json_list, ensure_ascii=False, indent=4)

# 保存为文件
with open("./wanfang_abstract.json", "w", encoding="utf-8") as f:
    f.write(json_data)

print("JSON 数据已保存到 wanfang_abstract.json")

JSON 数据已保存到 wanfang_abstract.json


In [10]:
import json
import random

# 读取数据
with open("./wanfang_abstract.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print("读取 JSON 数据成功")
print("原始数据长度:", len(data))

# 确保数据量足够进行抽样
if len(data) < 10000:
    print("数据量不足 10,000 条记录，无法进行抽样")
else:
    # 随机抽取 10,000 条记录
    sampled_data = random.sample(data, 10000)
    print("抽样完成，样本数据长度:", len(sampled_data))

    # 将抽样后的数据保存到新的 JSON 文件
    with open("./wanfang_abstract_sampled.json", "w", encoding="utf-8") as f:
        json.dump(sampled_data, f, ensure_ascii=False, indent=4)
    
    print("抽样数据已成功保存到 wanfang_abstract_sampled.json")

读取 JSON 数据成功
原始数据长度: 228836
抽样完成，样本数据长度: 10000
抽样数据已成功保存到 wanfang_abstract_sampled.json


In [5]:
import json
with open("./wanfang_abstract_sampled.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    print("读取 JSON 数据成功")
    print("数据长度:", len(data))
from openai import OpenAI
import dashscope
qa_client = OpenAI(base_url="http://127.0.0.1:8010/v1", api_key="sk-svcacct-Bi9jsZOB611YLuulKl1SfkKLwcpBHKU0-9U9Ztm3bylBuL_2i1DBMR04KYlGjGtT3BlbkFJ9tATCUjwUPLRY05FdX9v_mAg7gq7UoYv7smYmVaG9zUe4XY0pCxC7uIoDnNESAA")
def text_gen_chat_completion(
        prompt: str,
        model: str = "/home/lym/3l-dentalmind-backend/lllchat/model_files/backbone_model",
        max_tokens: int = 1024,
        temperature: float = 0.9) -> str:
    """Chat with the prompt."""
    completion = qa_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt},
        ],
        max_tokens=max_tokens,
        temperature=temperature,
    )

    return completion.choices[0].message.content
def call_large_model(prompt:str):
    messages=[{"role": "system", "content": "你是一个医学知识问答机器人。"},{"role": "user", "content": prompt}]
    response = dashscope.Generation.call(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key="sk-969f4200d53442a2a1733d1c0b1fb330",
    model="qwen-plus", # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    messages=messages,
    result_format='message'
    )
    return response['output']['choices'][0]['message']['content'].strip()
count=0
for i in data:
    count+=1
    if count%200==0:
        print(count)
    prompt=f"""请你从给定的论文题目中抽取包含的口腔疾病关键词，如果涉及请返回所有涉及实体(比如是口腔疾病相关实体)并用换行符分隔，如果没有涉及返回否，
    不要输出任何信息。
    示例：题目：老年口腔扁平苔藓患者半乳糖凝集素-3表达及临床意义。输出：口腔扁平苔藓\n题目:{i['题目']}"""
    response=call_large_model(prompt)
    if "否" not in response:
        print(i["题目"])
        disease_list=response.split("\n")
        print(disease_list)
        i["关联疾病"]=disease_list

读取 JSON 数据成功
数据长度: 228836
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
开窗引流术治疗牙源性颌骨囊肿的效果研究[J]
['牙源性颌骨囊肿']
锥形束CT(CBCT)在牙体牙髓病诊断治疗中的价值分析[J]
['牙体牙髓病']
维生素B12联合康复新液治疗口腔溃疡的价值[J]
['口腔溃疡']
牙周病伴牙列缺损修复治疗中活动义齿和固定义齿的应用效果分析[J]
['牙周病  ', '牙列缺损']
牙弓夹板用于急诊外伤脱位牙再植术后固定的价值及对美观效果的影响[J]
['牙脱位  ', '牙再植']
不同程度龋病儿童牙菌斑内微生物群落的差异性分析[J]
['龋病']
益生菌辅助盐酸米诺环素治疗中度慢性牙周炎的效果[J]
['慢性牙周炎']
大黄素对慢性牙周炎大鼠牙槽骨重建、炎症反应及RANKL/OPG蛋白表达的影响[J]
['慢性牙周炎']
荆门城区学龄前期儿童饮食习惯与患龋关系的研究[J]
['龋']
牙周基础治疗对牙周炎合并2型糖尿病患者相关指标的影响[J]
['牙周炎  ', '2型糖尿病']
超声骨刀技术应用于阻生牙拔除中的疗效[J]
['阻生牙']
复发性口腔溃疡的防治[J]
['复发性口腔溃疡']
牙髓血运重建术治疗年轻恒牙根尖周炎患者的效果[J]
['年轻恒牙根尖周炎']
石辛含片联合米诺环素治疗种植体周围炎的疗效及对龈沟液MMP-2、MMP-9水平的影响[J]
['种植体周围炎']
游离腓动脉穿支皮瓣在口腔颌面组织缺损修复中的研究进展[J]
['口腔颌面组织缺损']
头孢唑林和头孢呋辛分别联合奥硝唑治疗口腔颌面间隙感染的成本效果分析[J]
['口腔颌面间隙感染']
窝沟封闭术对预防儿童六龄齿发生龋病的临床效果分析[J]
['龋病']
罗定市青少年龋齿的患病现

Traceback (most recent call last):
  File "/home/lym/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2175211/3961769628.py", line 45, in <module>
    response=call_large_model(prompt)
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_2175211/3961769628.py", line 27, in call_large_model
    response = dashscope.Generation.call(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/lym/anaconda3/lib/python3.12/site-packages/dashscope/aigc/generation.py", line 138, in call
    response = super().call(model=model,
               ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/lym/anaconda3/lib/python3.12/site-packages/dashscope/client/base_api.py", line 145, in call
    return request.call()
           ^^^^^^^^^^^^^^
  File "/home/lym/anaconda3/lib/python3.12/site-packages/dashscope/api_entities/http_request.py", line 83, in call
    output = next(response)
   

In [4]:
data_new=[]
for j in data:
    if "关联疾病" in j.keys():
        data_new.append(j)
with open("./wanfang_abstract1_deduplicated.json", "r", encoding="utf-8") as f:
    data=json.load(f)
    print(len(data))
    print(len(data_new))
    data.extend(data_new)
    print(len(data))
with open("./wanfang_abstract1_deduplicated.json", "w", encoding="utf-8") as f:
    json.dump(data_new, f, ensure_ascii=False)

5202
887
6089


In [9]:
print(len(data))

887


In [8]:
with open("./wanfang_abstract1_deduplicated.json", "r", encoding="utf-8") as f:
    data=json.load(f)
with open("./wanfang_abstract1_deduplicated.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False,indent=4)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [1]:
import json

# 读取数据
with open("./wanfang_abstract1.json", "r", encoding="utf-8") as f:
    data = json.load(f)
print(len(data))
# 使用集合来存储已经见过的题目
seen_titles = set()
data_new = []

for record in data:
    title = record.get('题目', '').strip()
    if title not in seen_titles:
        seen_titles.add(title)
        data_new.append(record)
print(len(data_new))
# 将去重后的数据写回文件
with open("./wanfang_abstract1_deduplicated.json", "w", encoding="utf-8") as f:
    json.dump(data_new, f, ensure_ascii=False, indent=4)

print("数据已成功去重并写入 wanfang_abstract1_deduplicated.json")

5222
5202
数据已成功去重并写入 wanfang_abstract1_deduplicated.json


In [22]:
print(i)

{'来源': '万方', '作者': '王静', '题目': '对比腮腺部分切除术与腮腺浅叶切除术治疗腮腺良性肿瘤的效果[J]', '期刊信息': '健康必读,2023(15):36-37.', '作者部门': '金乡人民医院', '摘要': '目的:探讨腮腺部分切除术、腮腺浅叶切除术用于腮腺良性肿瘤治疗中的效果.方法:以本院收治的40例腮腺良性肿瘤患者为受试者(2021年1月至2023年1月),随机数表法分组,对照组行腮腺浅叶切除术治疗(20例),观察组行腮腺部分切除术治疗(20例),分析对比两组临床疗效.结果:观察组手术相关指标比对照组更优(P<0.05).术后两组生活质量均得到明显改善,但观察组评分更高(P<0.05).观察组并发症发生率、面神经损伤发生率比对照组更低(P<0.05).结论:在腮腺良性肿瘤治疗中,相比腮腺浅叶切除术治疗,腮腺部分切除术疗效更为突出,手术创伤程度较轻,且有助于降低患者术后并发症发生风险,利于患者术后恢复.'}
